# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("output_data/cities.csv")
city_data['City']=city_data['City'].str.title()
coordinates=city_data[['Lat', 'Lng']]
weight = city_data['Humidity']
max_intensity = city_data['Humidity'].max()
city_data.head()

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Provideniya,64.3833,-173.3000,57.31,62,0,8.95,RU,1624226543
1,1,Qaanaaq,77.4840,-69.3632,39.58,64,19,4.79,GL,1624226544
2,2,Yen Bai,21.7000,104.8667,81.54,72,66,2.59,VN,1624226544
3,3,San Andres,12.5847,-81.7006,82.80,79,45,17.05,CO,1624226544
4,4,Plast,54.3691,60.8136,50.27,49,1,11.43,RU,1624226545


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
figure_layout = {"width": "625","height": "500px","border": "1px solid black","padding": "1px","margin": "0 auto 0 auto"}
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(gmaps.heatmap_layer(coordinates, weights=weight, max_intensity=max_intensity, point_radius=6, opacity=0.8))
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

#Store into variable named hotel_df
hotel_df = pd.DataFrame(city_data.loc[((city_data["Max Temp"]>=70) & (city_data["Max Temp"]<=80))& (city_data["Wind Speed"]<10) & (city_data["Cloudiness"] == 0)])

# Add a "Hotel Name" column to the DataFrame.
hotel_df['HotelName'] = ""
hotel_df.head()

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
6,6,Cape Town,-33.9258,18.4232,70.09,12,0,9.22,ZA,1624226400,
35,35,Marrakesh,31.6315,-8.0083,73.47,43,0,9.22,MA,1624226554,
89,89,Nikolskoye,59.7035,30.7861,75.09,70,0,5.19,RU,1624226571,
189,189,Naqadeh,36.9553,45.3880,70.11,31,0,3.11,IR,1624226603,
270,270,Barcelona,41.3888,2.1590,76.69,69,0,1.01,ES,1624226625,


In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through hotel_df dataframe
# Set parameters to search each hotel within 5000 meters.
for index, row in hotel_df.iterrows():
    locations = str(row["Lat"])+","+str(row["Lng"])
    params = {
        "location": locations, 
        "radius": '5000',
        "type": "hotel",
        "key": g_key,
    }
    # Hit the Google Places API for each city's coordinates.
    response = requests.get(base_url, params=params).json()

    #Store the first Hotel result into the DataFrame.
    hotel_df.loc[index, 'HotelName'] = (response['results'][1]['name']).title()

In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt><b>Name</b></dt><dt>{HotelName}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df.head()

,Index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,HotelName
6,6,Cape Town,-33.9258,18.4232,70.09,12,0,9.22,ZA,1624226400,Southern Sun Waterfront Cape Town
35,35,Marrakesh,31.6315,-8.0083,73.47,43,0,9.22,MA,1624226554,Novotel Marrakech Hivernage
89,89,Nikolskoye,59.7035,30.7861,75.09,70,0,5.19,RU,1624226571,Nikolskaya College № 1
189,189,Naqadeh,36.9553,45.3880,70.11,31,0,3.11,IR,1624226603,دبیرستان غیردولتی خوارزمی
270,270,Barcelona,41.3888,2.1590,76.69,69,0,1.01,ES,1624226625,Axel Hotel Barcelona


In [38]:
# Plot markers on top of the heatmap.
markers = gmaps.marker_layer(locations, info_box_content=hotel_info,  hover_text='Click for Hotel')

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…